In [ ]:
%pip install numpy
%pip install pandas
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import pandas as pd

from ahc import AHC
from kmeans import K_Means

np.random.seed(5)

Carregando dataset Iris: https://archive.ics.uci.edu/dataset/53/iris

In [2]:
# dataset_path = input("Dataset path: ")
dataset_path = "./iris.csv"
data = pd.read_csv(dataset_path)
data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Ignorando coluna `class`

In [3]:
X = data.iloc[:, :4]
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Número de clusters

In [4]:
# k = int(input("Nro de clusters (k): "))
k = 3

Silhueta Simplificada

```
Para cada objeto i:
    Encontre o cluster ao qual i pertence
    Calcule a distância média intra-cluster (a_i):
        - Calcule a média das distâncias de i para todos os outros objetos no mesmo cluster
    Calcule a distância média mais próxima inter-cluster (b_i):
        - Para cada cluster diferente do cluster de i:
            - Calcule a média das distâncias de i para todos os objetos no cluster
            - Mantenha o mínimo dessas distâncias
    Calcule o valor da silhueta simplificada para o objeto i:
        - (b_i - a_i) / max(a_i, b_i) se o tamanho do cluster de i for maior que 1, caso contrário, 0
```

In [5]:
def simplified_silhouette(X, labels):
    n = X.shape[0]
    sil_scores = np.zeros(n)

    for i in range(n):
        # encontra o cluster ao qual i pertence
        own_cluster = labels[i]

        # a_i: distancia media intra-cluster,
        # media das distancias de i para todos os outros objs no mesmo cluster
        a_i = np.mean(np.linalg.norm(X[i] - X[labels == own_cluster], axis=1))

        # b_i: distancia media mais proxima inter-cluster,
        b_i = np.inf
        for label in np.unique(labels):
            if label == own_cluster:
                continue
            # para cada cluster diferente de i
            # 1. calcula a media das distancias de i para todos objetos no cluster
            # 2. calcula o minimo dessas distancias
            b_i = min(b_i, np.mean(np.linalg.norm(X[i] - X[labels == label], axis=1)))

        # calcula o valor da silhueta simplificada para i,
        # se o tamanho do cluster de i for maior que 1, caso contrario 0
        sil_scores[i] = (
            (b_i - a_i) / max(a_i, b_i) if np.sum(labels == own_cluster) > 1 else 0
        )

    return np.mean(sil_scores)

Validação dos algoritmos

In [6]:
model = K_Means(k=k)
kmeans_labels, kmeans_centroids = model.fit_predict(X.to_numpy())
kmeans_sil = simplified_silhouette(X.to_numpy(), kmeans_labels)

print("=== K-Means ===")
print("k\t\t", k)
print("Dataset\t\t", dataset_path)
print("Dataset shape\t", X.shape)
print("Silhouette Score", kmeans_sil)

=== K-Means ===
k		 3
Dataset		 ./iris.csv
Dataset shape	 (150, 4)
Silhouette Score 0.5471543677226165


In [7]:
ahc = AHC(X, k=k)
ahc.run()
ahc_sil = simplified_silhouette(X.to_numpy(), ahc.labels)

print("=== AHC ===")
print("k\t\t", k)
print("Dataset\t\t", dataset_path)
print("Dataset shape\t", X.shape)
print("Silhouette Score", ahc_sil)

=== AHC ===
k		 3
Dataset		 ./iris.csv
Dataset shape	 (150, 4)
Silhouette Score 0.517766369208731
